In [120]:
import pandas as pd
from Bio import SeqIO
from Bio.Data import CodonTable
from Bio.Seq import Seq
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [121]:
# important do not remove
for entry in SeqIO.parse('/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/misc_genbank/drosophila_melanogaster_mitogenome.gb', format='genbank'):
    for i in entry.features:
        if i.type == 'CDS':
            print(f"{entry.annotations['organism']} --- {i.location} --- {i.qualifiers['gene']}")
for entry in SeqIO.parse('/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/misc_genbank/periplaneta_brunnea_mitogenome.gb', format='genbank'):
    for i in entry.features:
        if i.type == 'CDS':
            print(f"{entry.annotations['organism']} --- {i.location} --- {i.qualifiers['gene']}")
    break

Drosophila melanogaster --- [239:1263](+) --- ['ND2']
Drosophila melanogaster --- [<1473:3009](+) --- ['COX1']
Drosophila melanogaster --- [3082:3767](+) --- ['COX2']
Drosophila melanogaster --- [3906:4068](+) --- ['ATP8']
Drosophila melanogaster --- [4061:4736](+) --- ['ATP6']
Drosophila melanogaster --- [4735:5524](+) --- ['COX3']
Drosophila melanogaster --- [5607:5961](+) --- ['ND3']
Drosophila melanogaster --- [6408:8125](-) --- ['ND5']
Drosophila melanogaster --- [8206:9545](-) --- ['ND4']
Drosophila melanogaster --- [9544:9835](-) --- ['ND4L']
Drosophila melanogaster --- [9970:10495](+) --- ['ND6']
Drosophila melanogaster --- [10498:11635](+) --- ['CYTB']
Drosophila melanogaster --- [11720:12659](-) --- ['ND1']
Periplaneta brunnea --- [223:1252](+) --- ['ND2']
Periplaneta brunnea --- [<1458:2997](+) --- ['COX1']
Periplaneta brunnea --- [3091:3776](+) --- ['COX2']
Periplaneta brunnea --- [3911:4070](+) --- ['ATP8']
Periplaneta brunnea --- [4063:4744](+) --- ['ATP6']
Periplaneta br

In [122]:
for entry in SeqIO.parse('/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/misc_genbank/drosophila_melanogaster_mitogenome.gb', format='genbank'):
    sps = []
    strands = []
    genes = []
    orders = []
    fams = []
    for i in entry.features:
        if i.type == 'CDS':
            order = entry.annotations['taxonomy'][9]
            fam = entry.annotations['taxonomy'][13]
            sp = entry.annotations['organism']
            if '+' in str(i.location):
                strand = '+'
            elif '-' in str(i.location):
                strand = '-'
            if 'gene' not in i.qualifiers:
                gene_name = 'NA'
            else:
                gene_name = ''.join(i.qualifiers['gene']).upper()
            sps.append(sp)
            strands.append(strand)
            genes.append(gene_name)
            orders.append(order)
            fams.append(fam)
    reference = pd.DataFrame([sps, orders, fams, strands, genes]).T
    reference = reference.rename(columns={0:'species', 1:'order', 2:'family', 3:'strand', 4:'gene'})
    reference = reference.sort_values(by=['gene'])
    reference = reference.reset_index(drop=True)

In [123]:
reference

,species,order,family,strand,gene
0,Drosophila melanogaster,Diptera,Drosophilidae,+,ATP6
1,Drosophila melanogaster,Diptera,Drosophilidae,+,ATP8
2,Drosophila melanogaster,Diptera,Drosophilidae,+,COX1
3,Drosophila melanogaster,Diptera,Drosophilidae,+,COX2
4,Drosophila melanogaster,Diptera,Drosophilidae,+,COX3
5,Drosophila melanogaster,Diptera,Drosophilidae,+,CYTB
6,Drosophila melanogaster,Diptera,Drosophilidae,-,ND1
7,Drosophila melanogaster,Diptera,Drosophilidae,+,ND2
8,Drosophila melanogaster,Diptera,Drosophilidae,+,ND3
9,Drosophila melanogaster,Diptera,Drosophilidae,-,ND4


In [124]:
different = pd.DataFrame()
similar = pd.DataFrame()
total_num = 0
for entry in SeqIO.parse('/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/misc_genbank/all_insects.gb', format='genbank'):
    sps = []
    strands = []
    genes = []
    taxonomy = []
    for i in entry.features:
        if i.type == 'CDS':
            tax = str(entry.annotations['taxonomy'][5:])
            sp = entry.annotations['organism']
            if '+' in str(i.location):
                strand = '+'
            elif '-' in str(i.location):
                strand = '-'
            if 'gene' not in i.qualifiers:
                gene_name = np.nan
            else:
                gene_name = ''.join(i.qualifiers['gene']).upper()

            #HAVE THEY FUCKING HEARD ABOUT STANDARTS?!
            if gene_name == 'COB':
                gene_name = 'CYTB'
            elif gene_name == 'COI' or gene_name == 'CO1' or gene_name == 'COXI':
                gene_name = 'COX1'
            elif gene_name == 'COII' or gene_name == 'CO2' or gene_name == 'COXII':
                gene_name = 'COX2'
            elif gene_name == 'COIII' or gene_name == 'CO3' or gene_name == 'COXIII':
                gene_name = 'COX3'
            elif gene_name == 'NAD1':
                gene_name = 'ND1'
            elif gene_name == 'NAD2':
                gene_name = 'ND2'
            elif gene_name == 'NAD3':
                gene_name = 'ND3'
            elif gene_name == 'NAD4':
                gene_name = 'ND4'
            elif gene_name == 'NAD4L':
                gene_name = 'ND4L'
            elif gene_name == 'NAD5':
                gene_name = 'ND5'
            elif gene_name == 'NAD6':
                gene_name = 'ND6'

            sps.append(sp)
            strands.append(strand)
            genes.append(gene_name)
            taxonomy.append(tax)
    df = pd.DataFrame([sps, taxonomy, strands, genes]).T
    df = df.rename(columns={0:'species', 1:'taxonomy', 2:'strand', 3:'gene'})
    df = df.sort_values(by=['gene'])
    df = df.reset_index(drop=True)
    df = df.dropna()
    if len(df) != 13: #check if species doesn't have all genes, in this case, drop 'em
        continue
    
    total_num += 1

    check = df[['strand', 'gene']] == reference[['strand', 'gene']] #outputs df with 2 cols of True/False values

    if check.sum().sum() != 26: #sum of all True vals is 26, if it's not equal 26, then we have False somewhere
        different = pd.concat([different, df])
    else:
        similar = pd.concat([similar, df])
different = different.sort_values(by=['taxonomy', 'species', 'gene'])
different.to_csv('/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/data/DescriptiveStat/insectsGB_with_diff_strand_loc.tsv', sep='\t')

/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/.venv/lib/python3.11/site-packages/Bio/GenBank/Scanner.py:1800: BiopythonParserWarning: Structured comment not parsed for NC_057214. Is it malformed?
  warnings.warn(
/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/.venv/lib/python3.11/site-packages/Bio/GenBank/Scanner.py:1800: BiopythonParserWarning: Structured comment not parsed for NC_057213. Is it malformed?
  warnings.warn(
/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/.venv/lib/python3.11/site-packages/Bio/GenBank/Scanner.py:1800: BiopythonParserWarning: Structured comment not parsed for NC_057212. Is it malformed?
  warnings.warn(
/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mutspec-insecta/.venv/lib/python3.11/site-packages/Bio/GenBank/Scanner.py:1800: BiopythonParserWarning: Structured comment not parsed for NC_057211. Is it malformed?
  warnings.warn(
/mnt/data/Documents/lab/TermitesAndCockroaches/mtdna-mut

In [125]:
len(different['species'].unique())

200

In [126]:
len(similar['species'].unique())

4661

In [127]:
total_num

4861